In [1]:
import pandas as pd
import numpy as np
import statsmodels.tsa.seasonal
from IPython.display import display
import plotly.express as px
import plotly.graph_objects as go

In [2]:
def run_sequence_plot(x, y, title, xtitle, ytitle):
    figure = px.line(x=x, y=y)
    figure.update_layout(title=title, width=800, xaxis_title=xtitle, yaxis_title=ytitle)
    return figure

# Read parquet dataset

In [5]:
df = pd.read_parquet("data/household.parquet")

# Exclude data before July 2007
df = df.loc["2007-07":]
df_monthly = df.resample("M").quantile(0.99)
var = "Global_active_power"
fig = run_sequence_plot(
    df_monthly.index, df_monthly[var], f"Monthly 99% percentile {var}", "Time", f"{var}"
)
fig.show()

# Cross-validation split

In [6]:
# Models
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima

In [9]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

tscv = TimeSeriesSplit(n_splits=4, test_size=3)

dates = []
y_real = []
y_pred_3sm = []
y_pred_autoarima = []
y_pred_sarima = []

for train, test in tscv.split(df_monthly.index):
    print(f"Train: {train}\nTest: {test}")

    triple_exp = ExponentialSmoothing(
        df_monthly[var].iloc[train], trend="add", seasonal="add", seasonal_periods=12
    ).fit(optimized=True)

    sar = SARIMAX(
        df_monthly[var].iloc[train],
        order=(1, 0, 0),
        seasonal_order=(0, 1, 1, 12),
        trend="t",
    ).fit()

    auto_model = auto_arima(
        df_monthly[var].iloc[train],
        start_p=0,
        start_q=0,
        max_p=3,
        max_q=3,
        m=12,
        start_P=0,
        seasonal=True,
        D=1,
        trace=True,
        error_action="ignore",
        suppress_warnings=True,
        stepwise=True,
    )

    dates.append(df_monthly.iloc[test].index)
    y_real.append(df_monthly[var].iloc[test])
    y_pred_sarima.append(sar.forecast(steps=len(test)))
    y_pred_autoarima.append(auto_model.predict(n_periods=len(test)))
    y_pred_3sm.append(triple_exp.forecast(len(test)))

Train: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28]
Test: [29 30 31]


c:\users\andre\pycharmprojects\householdconsumption\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.

c:\users\andre\pycharmprojects\householdconsumption\venv\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,1)[12] intercept   : AIC=inf, Time=0.15 sec
 ARIMA(0,0,0)(0,1,0)[12] intercept   : AIC=14.600, Time=0.01 sec
 ARIMA(1,0,0)(1,1,0)[12] intercept   : AIC=17.978, Time=0.17 sec
 ARIMA(0,0,1)(0,1,1)[12] intercept   : AIC=inf, Time=0.31 sec
 ARIMA(0,0,0)(0,1,0)[12]             : AIC=23.565, Time=0.02 sec
 ARIMA(0,0,0)(1,1,0)[12] intercept   : AIC=16.069, Time=0.05 sec
 ARIMA(0,0,0)(1,1,1)[12] intercept   : AIC=18.069, Time=0.12 sec
 ARIMA(1,0,0)(0,1,0)[12] intercept   : AIC=16.343, Time=0.02 sec
 ARIMA(0,0,1)(0,1,0)[12] intercept   : AIC=16.066, Time=0.03 sec
 ARIMA(1,0,1)(0,1,0)[12] intercept   : AIC=17.475, Time=0.04 sec

Best model:  ARIMA(0,0,0)(0,1,0)[12] intercept
Total fit time: 0.934 seconds
Train: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31]
Test: [32 33 34]


c:\users\andre\pycharmprojects\householdconsumption\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.



Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,1)[12] intercept   : AIC=20.519, Time=0.05 sec
 ARIMA(0,0,0)(0,1,0)[12] intercept   : AIC=19.604, Time=0.01 sec
 ARIMA(1,0,0)(1,1,0)[12] intercept   : AIC=22.503, Time=0.23 sec
 ARIMA(0,0,1)(0,1,1)[12] intercept   : AIC=22.490, Time=0.08 sec
 ARIMA(0,0,0)(0,1,0)[12]             : AIC=27.098, Time=0.01 sec
 ARIMA(0,0,0)(1,1,0)[12] intercept   : AIC=20.519, Time=0.03 sec
 ARIMA(0,0,0)(1,1,1)[12] intercept   : AIC=22.519, Time=0.05 sec
 ARIMA(1,0,0)(0,1,0)[12] intercept   : AIC=21.593, Time=0.03 sec
 ARIMA(0,0,1)(0,1,0)[12] intercept   : AIC=21.588, Time=0.03 sec
 ARIMA(1,0,1)(0,1,0)[12] intercept   : AIC=inf, Time=0.12 sec

Best model:  ARIMA(0,0,0)(0,1,0)[12] intercept
Total fit time: 0.637 seconds
Train: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34]
Test: [35 36 37]


c:\users\andre\pycharmprojects\householdconsumption\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.



Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,1)[12] intercept   : AIC=19.880, Time=0.10 sec
 ARIMA(0,0,0)(0,1,0)[12] intercept   : AIC=19.660, Time=0.02 sec
 ARIMA(1,0,0)(1,1,0)[12] intercept   : AIC=21.764, Time=0.08 sec
 ARIMA(0,0,1)(0,1,1)[12] intercept   : AIC=21.622, Time=0.10 sec
 ARIMA(0,0,0)(0,1,0)[12]             : AIC=28.603, Time=0.01 sec
 ARIMA(0,0,0)(1,1,0)[12] intercept   : AIC=19.880, Time=0.06 sec
 ARIMA(0,0,0)(1,1,1)[12] intercept   : AIC=21.880, Time=0.06 sec
 ARIMA(1,0,0)(0,1,0)[12] intercept   : AIC=21.590, Time=0.02 sec
 ARIMA(0,0,1)(0,1,0)[12] intercept   : AIC=21.551, Time=0.03 sec
 ARIMA(1,0,1)(0,1,0)[12] intercept   : AIC=inf, Time=0.10 sec

Best model:  ARIMA(0,0,0)(0,1,0)[12] intercept
Total fit time: 0.595 seconds
Train: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37]
Test: [38 39 40]


c:\users\andre\pycharmprojects\householdconsumption\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.

c:\users\andre\pycharmprojects\householdconsumption\venv\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,1)[12] intercept   : AIC=inf, Time=0.13 sec
 ARIMA(0,0,0)(0,1,0)[12] intercept   : AIC=23.053, Time=0.01 sec
 ARIMA(1,0,0)(1,1,0)[12] intercept   : AIC=21.616, Time=0.07 sec
 ARIMA(0,0,1)(0,1,1)[12] intercept   : AIC=inf, Time=0.17 sec
 ARIMA(0,0,0)(0,1,0)[12]             : AIC=33.100, Time=0.01 sec
 ARIMA(1,0,0)(0,1,0)[12] intercept   : AIC=25.028, Time=0.02 sec
 ARIMA(1,0,0)(2,1,0)[12] intercept   : AIC=inf, Time=0.42 sec
 ARIMA(1,0,0)(1,1,1)[12] intercept   : AIC=inf, Time=0.46 sec
 ARIMA(1,0,0)(0,1,1)[12] intercept   : AIC=inf, Time=0.36 sec
 ARIMA(1,0,0)(2,1,1)[12] intercept   : AIC=24.571, Time=0.39 sec
 ARIMA(0,0,0)(1,1,0)[12] intercept   : AIC=19.692, Time=0.04 sec
 ARIMA(0,0,0)(2,1,0)[12] intercept   : AIC=inf, Time=0.31 sec
 ARIMA(0,0,0)(1,1,1)[12] intercept   : AIC=inf, Time=0.48 sec
 ARIMA(0,0,0)(2,1,1)[12] intercept   : AIC=22.750, Time=0.37 sec
 ARIMA(0,0,1)(1,1,0)[12] intercept   : AIC=21.532, Time=0.06 sec
 AR

In [10]:
errors_df = pd.DataFrame(
    {
        "date": np.array(dates).flatten(),
        "real": np.array(y_real).flatten(),
        "sarima": np.array(y_pred_sarima).flatten(),
        "auto_arima": np.array(y_pred_autoarima).flatten(),
        "triple_smoothing": np.array(y_pred_3sm).flatten(),
    }
)

display(errors_df)

print(
    f"MAPE:\n"
    f"Triple Smoothing: {mean_absolute_percentage_error(errors_df.real, errors_df.triple_smoothing):.3f}\n"
    f"Auto ARIMA: {mean_absolute_percentage_error(errors_df.real, errors_df.auto_arima):.3f}\n"
    f"SARIMA: {mean_absolute_percentage_error(errors_df.real, errors_df.sarima):.3f}\n"
)

print(
    f"MSE:\n"
    f"Triple Smoothing: {mean_squared_error(errors_df.real, errors_df.triple_smoothing):.3f}\n"
    f"Auto ARIMA: {mean_squared_error(errors_df.real, errors_df.auto_arima):.3f}\n"
    f"SARIMA: {mean_squared_error(errors_df.real, errors_df.sarima):.3f}\n"
)

,date,real,sarima,auto_arima,triple_smoothing
0,2009-12-31,5.09044,5.050850,4.728644,5.025078
1,2010-01-31,4.87406,4.937716,5.209424,5.004891
2,2010-02-28,4.84084,3.981869,4.147424,4.079830
3,2010-03-31,4.51406,4.381310,4.287417,4.343415
4,2010-04-30,4.01282,4.162653,4.141457,4.221857
5,2010-05-31,4.11602,4.127686,3.912637,4.218602
6,2010-06-30,4.03282,3.818071,3.490541,3.900628
7,2010-07-31,3.41000,3.761942,3.834521,3.827244
8,2010-08-31,3.97004,3.920734,4.152521,4.005775
9,2010-09-30,3.87928,4.008781,4.031478,4.088827


MAPE:
Triple Smoothing: 0.055
Auto ARIMA: 0.075
SARIMA: 0.050

MSE:
Triple Smoothing: 0.094
Auto ARIMA: 0.134
SARIMA: 0.095



In [ ]:
figure = px.line(x=df_monthly.index, y=df_monthly[var])
figure.add_scatter(x=errors_df.date, y=errors_df.sarima, mode="lines", name="sarima")
figure.add_scatter(
    x=errors_df.date, y=errors_df.auto_arima, mode="lines", name="auto arima"
)
figure.add_scatter(
    x=errors_df.date, y=errors_df.triple_smoothing, mode="lines", name="3x smoothing"
)
figure.update_layout(
    title=f"Monthly 99% percentile {var}",
    width=1000,
    xaxis_title="Time",
    yaxis_title=f"{var}",
)
figure.show()